# Evaluate using test set of GDUT-HWD dataset (hardhat wearing detection)

In [ ]:
from yolo import YOLO
from PIL import Image

from keras_utils import get_classes, get_anchors
from yolo3.utils import get_data
from yolo3_utils import *

np.set_printoptions(precision=2)

In [ ]:
#data_mode = 1 #original
data_mode = 2 #binary (2 classes: hardhat, none)

In [ ]:
# weights_path = 'model_data/yolo.h5'
if data_mode == 1:
    classes_path = 'model_data/GDUT-HWD_classes.txt'
    weights_path = 'model_data/GDUT-HWD_trained_weights_final.h5'
    test_annotation_path = 'Hardhat_test.txt'
elif data_mode == 2:
    classes_path = 'model_data/GDUT-HWD_binary_classes.txt'
    weights_path = 'model_data/GDUT-HWD_binary_trained_weights_final.h5'
    test_annotation_path = 'Hardhat_binary_test.txt'
#     test_annotation_path = 'Hardhat_binary_test_small.txt'

In [ ]:
anchors_path = 'model_data/yolo_anchors.txt'
class_names = get_classes(classes_path)
num_classes = len(class_names)
anchors = get_anchors(anchors_path)

input_shape = (416,416) # multiple of 32, hw

In [ ]:
yolo_ = YOLO(model_path=weights_path, classes_path=classes_path, anchors_path=anchors_path)
print(yolo_.class_names)

## Evaluate one image

In [ ]:
# image_id = '01422'
# image_id = '02160'
# image_id = '02475'
# image_id = '02190'
# image_id = '00671'
image_id = '02464'
photo_filename = '/home/krittametht/data/Hardhat/JPEGImages/' + image_id + '.jpg'
print(photo_filename)

show_image = False
# show_image = True

try:
    image = Image.open(photo_filename)
except:
    print('Open Error! Try again!')
else:
    r_image, out_boxes, out_scores, out_classes = yolo_.detect_image(image, print_log=True)
    if show_image:
        r_image.show()

# print(out_boxes)
# print(out_scores)
# print(out_classes)

In [ ]:
# find the line with image id in txt file
out = ! grep -n $image_id".jpg" $test_annotation_path
if len(out) == 1:
    parts = out[0].split(':')
    line_annotation = parts[1]

actual_box = get_data(line_annotation)

In [ ]:
nonzero_actual_boxes = np.asarray([b for b in actual_box if not all_equals(b)], dtype=np.float32)
actual_classes = nonzero_actual_boxes[:,4]
actual_boxes = nonzero_actual_boxes[:,:4]

out_boxes_2 = copy_and_swap_columns(out_boxes, [1, 0, 3, 2])

print_log = False
print_log = True
# iou_threshold = 0.5
iou_threshold = 0.75
acc = classification_accuracy_one_image(actual_boxes, out_boxes_2, actual_classes, out_classes, print_log=print_log, iou_threshold=iou_threshold)

print(actual_classes)
print(out_classes)

print(acc)

## Evaluate multiple images

In [ ]:
with open(test_annotation_path) as f:
    lines = f.readlines()

In [ ]:
# based on data_generator
def generate_data(annotation_lines):
    n = len(annotation_lines)
    i = 0
    while i < n:
        box = get_data(annotation_lines[i])
        image_file = annotation_lines[i].split(' ')[0]
        i = i+1
        yield image_file, box

# iou_threshold = 0.5
iou_threshold = 0.75
show_image = False
accs = []
log_every_n_iterations = 100
itr = 0
for photo_filename, actual_box in generate_data(lines):
    try:
        image = Image.open(photo_filename)
    except:
        print('Error: Open Image File')
    else:
        r_image, out_boxes, out_scores, out_classes = yolo_.detect_image(image, print_log=False)
        if show_image: r_image.show()
    
        nonzero_actual_boxes = np.asarray([b for b in actual_box if not all_equals(b)], dtype=np.float32)
        actual_classes = nonzero_actual_boxes[:,4]
        actual_boxes = nonzero_actual_boxes[:,:4]

        out_boxes_2 = copy_and_swap_columns(out_boxes, [1, 0, 3, 2])

        acc = classification_accuracy_one_image(actual_boxes, out_boxes_2, actual_classes, out_classes, print_log=False, iou_threshold=iou_threshold)
        accs.append(acc)
        
        itr = itr + 1
        if itr % log_every_n_iterations == 0:
            print(itr, accs)

In [ ]:
np_accs = np.asarray(accs)
print(np_accs.mean())
print(len(accs), accs)